In [ ]:
%matplotlib inline

import numpy as np
import os
import matplotlib.pyplot as plt

from bayeso import gp
from bayeso import covariance
from bayeso.utils import utils_covariance

In [ ]:
num_points = 1000
str_cov = 'se'
int_init = 1
int_iter = 50
int_ts = 100

list_Y_min = []

X = np.expand_dims(np.linspace(-5, 5, num_points), axis=1)
mu = np.zeros(num_points)
hyps = utils_covariance.get_hyps(str_cov, 1)
Sigma = covariance.cov_main(str_cov, X, X, hyps, True)

In [ ]:
for ind_ts in range(0, int_ts):
    print('TS:', ind_ts + 1, 'iteration')
    Y = gp.sample_functions(mu, Sigma, num_samples=1)[0]

    ind_init = np.argmin(Y)
    bx_min = X[ind_init]
    y_min = Y[ind_init]
    
    ind_random = np.random.choice(num_points)

    X_ = np.expand_dims(X[ind_random], axis=0)
    Y_ = np.expand_dims(np.expand_dims(Y[ind_random], axis=0), axis=1)

    for ind_iter in range(0, int_iter):
        print(ind_iter + 1, 'iteration')

        mu_, sigma_, Sigma_ = gp.predict_optimized(X_, Y_, X, str_cov=str_cov)
        ind_ = np.argmin(gp.sample_functions(np.squeeze(mu_, axis=1), Sigma_, num_samples=1)[0])

        X_ = np.concatenate([X_, [X[ind_]]], axis=0)
        Y_ = np.concatenate([Y_, [[Y[ind_]]]], axis=0)
        
    list_Y_min.append(Y_ - y_min)

Ys = np.array(list_Y_min)
Ys = np.squeeze(Ys, axis=2)
print(Ys.shape)

In [ ]:
plt.plot(np.arange(0, 51), np.mean(Ys, axis=0), lw=4)
plt.fill_between(np.arange(0, 51),
                 np.mean(Ys, axis=0) - 0.2 * np.std(Ys, axis=0),
                 np.mean(Ys, axis=0) + 0.2 * np.std(Ys, axis=0),
                 alpha=0.3)